In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

Using TensorFlow backend.


In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [3]:
TRAINING_SIZE = 80000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789+ '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)

In [6]:
ctable.indices_char

{0: ' ',
 1: '+',
 2: '0',
 3: '1',
 4: '2',
 5: '3',
 6: '4',
 7: '5',
 8: '6',
 9: '7',
 10: '8',
 11: '9'}

In [7]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 80000


In [8]:
print(questions[:5], expected[:5])

['20+552 ', '159+35 ', '6+533  ', '64+1   ', '694+601'] ['572 ', '194 ', '539 ', '65  ', '1295']


In [9]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [10]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:20000]
train_y = y[:20000]
test_x = x[20000:]
test_y = y[20000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(18000, 7, 12)
(18000, 4, 12)
Validation Data:
(2000, 7, 12)
(2000, 4, 12)
Testing Data:
(60000, 7, 12)
(60000, 4, 12)


In [17]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False  True False False False False False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False False  True False False False
   False]
  [False False  True False False False False False False False False
   False]
  [False False False  True False False False False False False False
   False]
  [ True False False False False False False False False False False
   False]
  [ True False False False False False False False False False False
   False]]

 [[False False False  True False False False False False False False
   False]
  [False False False False False False False False  True False False
   False]
  [False False False False False  True False False False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False False False False  True False
   False]
  [False False False False False False Fal

In [12]:
print('Build model...')
model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [13]:
for iteration in range(10):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=10,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Instructions for updating:
Use tf.cast instead.
Train on 18000 samples, validate on 2000 samples
Epoch 1/10
18000/18000 [==============================] - 4s 227us/step - loss: 2.0180 - acc: 0.2886 - val_loss: 1.8628 - val_acc: 0.3310
Epoch 2/10
18000/18000 [==============================] - 3s 171us/step - loss: 1.8453 - acc: 0.3312 - val_loss: 1.8184 - val_acc: 0.3324
Epoch 3/10
18000/18000 [==============================] - 3s 171us/step - loss: 1.8083 - acc: 0.3325 - val_loss: 1.7943 - val_acc: 0.3291
Epoch 4/10
18000/18000 [==============================] - 4s 195us/step - loss: 1.7745 - acc: 0.3427 - val_loss: 1.7460 - val_acc: 0.3519
Epoch 5/10
18000/18000 [==============================] - 3s 184us/step - loss: 1.7254 - acc: 0.3591 - val_loss: 1.6931 - val_acc: 0.3625
Epoch 6/10
18000/18000 [==============================] - 3s 186us/step - loss: 1.6625 - acc: 0.3757 - val_loss: 1.6252 - val_acc: 0.3828
Epoch 7/10


Epoch 5/10
18000/18000 [==============================] - 3s 142us/step - loss: 0.1569 - acc: 0.9671 - val_loss: 0.1834 - val_acc: 0.9494
Epoch 6/10
18000/18000 [==============================] - 3s 152us/step - loss: 0.1299 - acc: 0.9786 - val_loss: 0.1598 - val_acc: 0.9594
Epoch 7/10
18000/18000 [==============================] - 3s 185us/step - loss: 0.1248 - acc: 0.9787 - val_loss: 0.1618 - val_acc: 0.9576
Epoch 8/10
18000/18000 [==============================] - 3s 148us/step - loss: 0.1278 - acc: 0.9749 - val_loss: 0.3336 - val_acc: 0.8986
Epoch 9/10
18000/18000 [==============================] - 3s 164us/step - loss: 0.1383 - acc: 0.9675 - val_loss: 0.1284 - val_acc: 0.9710
Epoch 10/10
18000/18000 [==============================] - 3s 175us/step - loss: 0.0935 - acc: 0.9870 - val_loss: 0.1269 - val_acc: 0.9689
Q 65+44   T 109  ☑ 109 
Q 579+241 T 820  ☑ 820 
Q 5+843   T 848  ☑ 848 
Q 842+73  T 915  ☑ 915 
Q 9+792   T 801  ☒ 891 
Q 668+793 T 1461 ☑ 1461
Q 770+13  T 783  ☑ 783 
Q 6

18000/18000 [==============================] - 4s 233us/step - loss: 0.0618 - acc: 0.9803 - val_loss: 0.4491 - val_acc: 0.8790
Epoch 2/10
18000/18000 [==============================] - 4s 236us/step - loss: 0.0916 - acc: 0.9719 - val_loss: 0.0576 - val_acc: 0.9811
Epoch 3/10
18000/18000 [==============================] - 4s 236us/step - loss: 0.0144 - acc: 0.9983 - val_loss: 0.0400 - val_acc: 0.9884
Epoch 4/10
18000/18000 [==============================] - 4s 233us/step - loss: 0.0092 - acc: 0.9995 - val_loss: 0.0378 - val_acc: 0.9891
Epoch 5/10
18000/18000 [==============================] - 4s 225us/step - loss: 0.0078 - acc: 0.9998 - val_loss: 0.0364 - val_acc: 0.9890
Epoch 6/10
18000/18000 [==============================] - 4s 230us/step - loss: 0.0074 - acc: 0.9998 - val_loss: 0.0366 - val_acc: 0.9888
Epoch 7/10
18000/18000 [==============================] - 4s 235us/step - loss: 0.0068 - acc: 0.9998 - val_loss: 0.0356 - val_acc: 0.9888
Epoch 8/10
18000/18000 [=====================

In [15]:
Right = 0
Wrong = 0
for i in range(10000):
    ind = np.random.randint(0, len(test_x))
    rowx, rowy = test_x[np.array([i])], test_y[np.array([i])]
    preds = model.predict_classes(rowx, verbose=0)
    q = ctable.decode(rowx[0])
    correct = ctable.decode(rowy[0])
    guess = ctable.decode(preds[0], calc_argmax=False)
    
    #print('Q', q[::-1] if REVERSE else q, end=' ')
    #print('T', correct, end=' ')

    if correct == guess:
        Right += 1
    else:
        Wrong += 1
print(Right/10000)

0.9587


In [48]:
len(test_x)

60000